# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 比较削减风场前后的MCIP数据结果

---
*@author: Evan*\
*@date: 2023-05-15*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
uvwind = xr.open_dataset('F:/Data/case_anqing/April/COMBINE_ACONC_CN3AH_135X138_2023_uvwind.nc')
met = xr.open_dataset('F:/Data/case_anqing/April/COMBINE_ACONC_CN3AH_135X138_2023_met.nc')
grid = xr.open_dataset('F:/Data/case_anqing/GRIDCRO2D_2023021.nc')

In [3]:
times=pd.date_range('2023-04-06-00','2023-04-10-23',freq='h')
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50

In [4]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from mcip
        PRES=(['time','level','y','x'],met.PRES[days*24-8:-8-1,:21,:,:].data,{'long name':'Air Pressure','units':'hPa'}),
        WSPD10=(['time','level','y','x'],met.WSPD10[days*24-8:-8-1,:21,:,:].data,{'long name':'Wind Speed 10m','units':'m s-1'}),
        WDIR10=(['time','level','y','x'],met.WDIR10[days*24-8:-8-1,:21,:,:].data,{'long name':'Wind Direction','units':'deg'}),
        uwind=(['time','level','y','x'],uvwind.UWind[days*24-8:-8-1,:21,:-1,:-1].data,{'long name':'U-direction Horizontal Wind Speed','units':'m s-1'}),
        vwind=(['time','level','y','x'],uvwind.VWind[days*24-8:-8-1,:21,:-1,:-1].data,{'long name':'V-direction Horizontal Wind Speed','units':'m s-1'}),
        ),
    coords=dict(
        time=times,
        level=pres[:21],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202304',
        grid='CN3AH_135X138',
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 120, level: 21, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-04-06 ... 2023-04-10T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 859.5 833.8 804.0
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    PRES       (time, level, y, x) float32 971.2 940.8 917.6 ... 790.6 790.6
    WSPD10     (time, level, y, x) float32 1.621 2.054 1.979 ... 0.0 0.0 0.0
    WDIR10     (time, level, y, x) float32 28.88 45.49 48.49 ... 0.0 0.0 0.0
    uwind      (time, level, y, x) float32 -0.3227 -1.11 -1.89 ... 3.709 3.734
    vwind      (time, level, y, x) float32 -1.576 -1.688 -1.388 ... 3.887 3.845
Attributes:
    case:     Anqing_202304
    grid:     CN3AH_135X138

In [5]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/April_05wind.nc',encoding=encoding)